# Build a recommender

*by [Longqi@Cornell](http://www.cs.cornell.edu/~ylongqi/) licensed under [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/)*

This tutorial demonstrates the process of building recommendation algorithms using OpenRec. For training and evaluation, please refer to [tutorial-Get Started](https://github.com/ylongqi/openrec/blob/master/tutorials/OpenRec%20Tutorial%20%231.ipynb).

Building a recommender takes two steps:
* Decompose a recommendation algorithm into **modules** + **a computational graph**.
* Inherent from the [Recommender](http://openrec.readthedocs.io/en/latest/recommenders/openrec.recommenders.recommender.html) class and overwrite corresponding functions.

In this tutorial, we will start by building a simple [PMF (Probablistic Matrix Factorization)](https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf) recommender and then extend it to a VisualPMF recommender that additionally incorporates visual signals (e.g., leveraging product images for recommendation).

## Probablistic Matrix Factorization (PMF)

### 1. Decompose PMF into modules + a computational graph
PMF can be represented as the following computational graph, and it consists of two reusable components:

<img src="https://s3.amazonaws.com/cornell-tech-sdl-openrec/files/pmf_flow.png" width="200px">

* Extraction: embeds each user and item id into a vector, and 
* Interaction: uses dot-product and Mean Square Error (MSE) to model interactions (implicit feedback),

### 2. Implementation using OpenRec *Recommender*
To implement the modularized PMF, we first map the prior defined components into OpenRec modules, as shown below.

<img src="https://s3.amazonaws.com/cornell-tech-sdl-openrec/files/openrec_pmf.png" width="200px">

Then we sequentially define inputs, input mappings, extraction modules, and interaction modules as [specified in the Recommender](http://openrec.readthedocs.io/en/latest/recommenders/openrec.recommenders.recommender.html). Functions `add_input` and `add_module` should be used to construct inputs/modules. To connect modules and inputs together to form a computational graph, functions `get_input` and `get_module` should be used to retrieve existing inputs/modules.

(Note that in the implementation below, we additionally add an **item bias** to capture popularity signals)

In [ ]:
class PMF(Recommender):

    def __init__(self, batch_size, dim_embed, max_user, max_item,
                    test_batch_size=None, l2_reg=None, opt='SGD', sess_config=None):

        self._dim_embed = dim_embed

        super(PMF, self).__init__(batch_size=batch_size, 
                                  test_batch_size=test_batch_size,
                                  max_user=max_user, 
                                  max_item=max_item, 
                                  l2_reg=l2_reg,
                                  opt=opt, sess_config=sess_config)

    '''
    Define inputs using `add_input` function
    '''
    def _build_user_inputs(self, train=True):
        
        if train:
            self._add_input(name='user_id', dtype='int32', shape=[self._batch_size])
        else:
            self._add_input(name='user_id', dtype='int32', shape=[None], train=False)
    
    def _build_item_inputs(self, train=True):
        
        if train:
            self._add_input(name='item_id', dtype='int32', shape=[self._batch_size])
        else:
            self._add_input(name='item_id', dtype='none', train=False)
    
    def _build_extra_inputs(self, train=True):
        
        if train:
            self._add_input(name='lables', dtype='float32', shape=[self._batch_size])
    
    '''
    Define input mappings
    '''
    def _input_mappings(self, batch_data, train):

        if train:
            return {self._get_input('user_id'): batch_data['user_id_input'],
                    self._get_input('item_id'): batch_data['item_id_input'],
                    self._get_input('labels'): batch_data['labels']}
        else:
            return {self._get_input('user_id', train=False): batch_data['user_id_input']}

    '''
    Define modules using `add_module` function.
    '''
    def _build_user_extractions(self, train=True):

        self._add_module('user_vec', 
                         LatentFactor(l2_reg=self._l2_reg, init='normal', 
                                      ids=self._get_input('user_id', train=train),
                                      shape=[self._max_user, self._dim_embed], 
                                      scope='user', reuse=not train), 
                         train=train)
    
    def _build_item_extractions(self, train=True):
        
        self._add_module('item_vec',
                         LatentFactor(l2_reg=self._l2_reg, init='normal', 
                                      ids=self._get_input('item_id', train=train),
                                      shape=[self._max_item, self._dim_embed], 
                                      scope='item', reuse=not train), 
                         train=train)
        self._add_module('item_bias',
                         LatentFactor(l2_reg=self._l2_reg, init='zero', 
                                      ids=self._get_input('item_id', train=train),
                                      shape=[self._max_item, 1], 
                                      scope='item_bias', reuse=not train), 
                         train=train)

    def _build_default_interactions(self, train=True):

        self._add_module('interaction',
                    PointwiseMSE(user=self._get_module('user_vec', train=train).get_outputs()[0], 
                                 item=self._get_module('item_vec', train=train).get_outputs()[0],
                                 item_bias=self._get_module('item_bias', train=train).get_outputs()[0], 
                                 labels=self._get_input('labels'), a=1.0, b=1.0, 
                                 train=train, scope='PointwiseMSE', reuse=not train),
                        train=train)

    def _build_serving_graph(self):

        super(PMF, self)._build_serving_graph()
        self._scores = self._get_module('interaction', train=False).get_outputs()[0]

## Visual Probablistic Matrix Factorization (VisualPMF)

Built upon PMF, VisualPMF additionally incorporates visual signals with the following structure (left). The corrsponding OpenRec modules are shown on the right.

<img src="https://s3.amazonaws.com/cornell-tech-sdl-openrec/files/visual_pmf.png" width="700px">

Because PMF is highly modular, VisualPMF can be impolemented by directly inherenting from PMF and extending functions `build_item_inputs`, `input_mappings`, `build_item_interactions`, and `build_default_fusions`.

In [ ]:
class VisualPMF(PMF):

    def __init__(self, batch_size, max_user, max_item, dim_embed, dims, item_f_source, 
                 test_batch_size=None, item_serving_size=None, dropout_rate=None,
                l2_reg_u=None, l2_reg_mlp=None, l2_reg_v=None, opt='SGD', sess_config=None):

        self._dims = dims
        self._dropout_rate = dropout_rate
        self._item_f_source = item_f_source
        self._item_serving_size = item_serving_size

        self._l2_reg_u = l2_reg_u
        self._l2_reg_mlp = l2_reg_mlp
        self._l2_reg_v = l2_reg_v

        super(VisualPMF, self).__init__(batch_size=batch_size, 
                                        max_user=max_user, 
                                        max_item=max_item, 
                                        dim_embed=dim_embed,
                                        test_batch_size=test_batch_size, 
                                        opt=opt, sess_config=sess_config)

    def _build_item_inputs(self, train=True):
        
        super(VisualPMF, self)._build_item_inputs(train)
        if train:
            self._add_input(name='item_vfeature', dtype='float32', 
                            shape=[self._batch_size, self._item_f_source.shape[1]])
        else:
            self._add_input(name='item_id', dtype='int32', shape=[None], train=False)
            self._add_input(name='item_vfeature', dtype='float32', 
                            shape=[None, self._item_f_source.shape[1]], train=False)

    def _input_mappings(self, batch_data, train):

        default_input_map = super(VisualPMF, self)._input_mappings(batch_data=batch_data, 
                                                                   train=train)
        if train:
            default_input_map[self._get_input('item_vfeature')] = \
                            self._item_f_source[batch_data['item_id_input']]
        else:
            default_input_map[self._get_input('item_id', train=False)] = \
                            batch_data['item_id_input']
            default_input_map[self._get_input('item_vfeature', train=False)] = \
                            self._item_f_source[batch_data['item_id_input']]
        return default_input_map

    def _build_item_extractions(self, train=True):

        super(VisualPMF, self)._build_item_extractions(train)
        self._add_module('item_vf',
                         MultiLayerFC(in_tensor=self._get_input('item_vfeature', train=train), 
                                      dims=self._dims, l2_reg=self._l2_reg_mlp, 
                                      dropout_mid=self._dropout_rate, train=train,
                                      scope='item_MLP', reuse=not train),
                         train=train)

    def _build_default_fusions(self, train=True):

        self._add_module('item_vec',
                        Average(scope='item_average', reuse=not train, 
                                module_list=[self._get_module('item_vec', train=train), 
                                self._get_module('item_vf', train=train)], weight=2.0),
                        train=train)